# MACHINE LEARNING FOR FINANCIAL SERVICES 

Welcome to IBM's Data Science Experience! This exciting tool will help your life a lot easier as a data scientist.  Below is a simple introductory example of how easy for you to load your data and run some simple regression models.

## Simple Linear Regression & Supervised Learning for Logistic Regression

Regression mode is a quite popular statistical modeling technique as it is intuitive to understand its concept by showing relationship between attributes. There are two major types of regression models: linear and logistic regressions.  Linear regression is designed to show linear relationship between independent and dependent variables, while logistic regression is designed for categorical dependent variable. Logistic regression, despite its name, is a linear model for classification rather than regression.  In spite of its simple concept, the relationship should be carefully considered as it does not necessarily show causality, but rather correlation.

### Scikit Learn

from [wikipedia](https://en.wikipedia.org/wiki/Scikit-learn)

>Scikit-learn (formerly scikits.learn) is a free software machine learning library for the Python programming language. It features various classification, regression and clustering algorithms including support vector machines, random forests, gradient boosting, k-means and DBSCAN, and is designed to interoperate with the Python numerical and scientific libraries NumPy and SciPy.  It also includes a few rudimentary deep learning packages as well.  I plan to utilize Scikit Learn library for many of my illustrative examples as it is well-documented and well-used in Python community.

## To load the data:

1. Load your local file into your notebook. Click the **Find and Add Data** icon on the notebook action bar. Drop the file into the box or browse to select the file. The file is loaded to your object storage and appears in the Data Assets section of the project. For more information, see <a href="https://datascience.ibm.com/docs/content/analyze-data/load-and-access-data.html" target="_blank" rel="noopener noreferrer">Load and access data</a>.
1. click in the next code cell and select **Insert to code > pandas DataFrame** under the file name.

In [ ]:
from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_fcf1c90868844bc1ab7d4fffe6063140(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_4a6fd2c26617760f817f90c4ebbaee820653e676','domain': {'id': 'xxxx Your ID xxxx'},
            'password': 'xxxx Your Passwordxxx'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df = pd.read_csv(get_object_storage_file_with_credentials_fcf1c90868844bc1ab7d4fffe6063140('MLshowcase', 'CRPMT_SAMPLE.csv'))
df.describe()

## Attributes in the sample file 

> this is a hypothetical sample file from a credit card company, in which a few attributes are mocked up for the various aspects of credit/payment business such as risk management, contact strategy and CRM

1. ACCT_NO: Account Number
1. PROD: Product Tier (1.REG: regular, 2.GOLD: gold and 3.PLAT: platinum cards)
1. CURR_BAL: Current balance on the account
1. TENURE: # of years since the account opened
1. CUST_AGE: Customer's age in years
1. PMT_DUE: Due amount on current billing cycle
1. NO_DM_CNT: # of Direct Mail Contacts for the last 2 years
1. WRITE_OFF_IND: Write-Off Indicator (0: Not written-off, 1: Written-Off)
1. FICO_SCR: Fair-Issac Score or FICO (a credit score that is being calculated by US credit bureau such as Experian, Equifax or TransUnion)

## SIMPLE LINEAR REGRESSION

## Are you sending junk mails to your profitable customers?

In this section, I'd like to show a very simple linear regression relationship by taking a sample credit card data.  Across different industry sectors where direct customer and prospect marketing is necessary, this became a typical and annoying challenge for many companies and consumers as their marketing contact strategy was specifically concentrated on profitable customer segments.  In this sample credit card data, the higher credit-worthy customers were, the more pieces of direct mails were sent.  This led to "adverse selection" (meaning those responded the marketing contacts are usually borderline or marginally valued marketing segments) or to attrition or marketing opt-outs from the highly profitable customers.  

In [ ]:
# create a feature, FICO SCORE, and a response, # OF DIRECT MAIL CONTACTS, vectors

X = df.FICO_SCR
y = df.NO_DM_CNT


# reshape X and y to the single array of the attributes as follows

X = X.reshape(-1, 1)
y = y.reshape(-1, 1)

In [ ]:
# running a linear regression model

from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(X, y)

In [ ]:
# print the intercept and coefficients
print(linreg.intercept_)
print(linreg.coef_)

## Visualizing Linear Regression 

In [ ]:
# import Matplotlib (scientific plotting library)
import matplotlib.pyplot as plt

# allow plots to appear within the notebook
%matplotlib inline

# Plot true values versus the predictions
plt.scatter(df.FICO_SCR, df.NO_DM_CNT, color="b")
plt.xlabel("FICO SCORE")
plt.ylabel("# of Direct Mail contacts")
plt.show()

In [ ]:
# Quick visualization of data relationships using seaborn visualization API
import seaborn as sns
sns.set_style('whitegrid')

# Using seabron to create a linear fit
sns.lmplot('FICO_SCR','NO_DM_CNT', data = df)

## CREDIT DEFAULT PREDICTION 

### LOGSTIC REGRESSION MODEL SETUP, TRAINING & ACCURACY CHECK

#### Below is an illustrative example of finding out how write-off response may be correlated with predictor (feature) variables by training a multi-variate logistic regression model

In [ ]:
# select a few numerical variables (predictors)

feature_cols = [          
    'CURR_BAL',                                               
    'TENURE',                       
    'CUST_INC',                      
    'CUST_AGE',                                
    'PMT_DUE',                                               
    'NO_DM_CNT',               
    'FICO_SCR',
    'PROD_NO'
]

In [ ]:
# create a feature matrix 'X' 

X = df[feature_cols]
X.shape

#print (type(X))
# X.head()

In [ ]:
# create a response vector series 'y' 
y = df.WRITE_OFF_IND
y.shape
#y.dtypes

In [ ]:
# if you need to upgrade scikit learn
#!pip install --upgrade scikit-learn

In [ ]:
# splitting in-sample training and test datasets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
# default split is 75% for training and 25% for testing; but chose above test size (test_size)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# import and instantiate a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

# train the model
logreg.fit(X_train, y_train)

In [ ]:
# make class predictions for X_test
# as there are a large number of zero values, the predictions are also skewed into zeros as well

y_pred_class = logreg.predict(X_test)
y_pred_class

In [ ]:
# calculate predicted probabilities for X_test (well calibrated)
y_pred_prob = logreg.predict_proba(X_test)[:, 1]
y_pred_prob

In [ ]:
import sklearn.metrics as metrics

# calculate accuracy
print 'Accuracy score is: {}'.format(metrics.accuracy_score(y_test, y_pred_class))

In [ ]:
# calculate AUC
print 'AUC is: {}'.format(metrics.roc_auc_score(y_test, y_pred_prob))

#### ROC curve allows to see how sensitivity and specificity are affected by various thresholds, and AUC is the percentage of the ROC plot that is underneath the curve

- AUC is useful as a single number summary of classifier performance.
- If you randomly chose one positive and one negative observation, AUC represents the likelihood that your classifier will assign a higher predicted probability to the positive observation.
- AUC is useful even when there is high class imbalance (unlike classification accuracy).